## Lattice constant calculation with Conquest

In [ ]:
import os

In [ ]:
basis_size = 'medium'
xc = 'PBE'

basis = {
         'Cu': {'gen_basis'            : True,
                'basis_size'           : basis_size,
                "pseudopotential_type": 'hamann',
                'xc'                   : xc,
                },
         'O' : { 'gen_basis'            : True,
                 'basis_size'           : basis_size,
                 'pseudopotential_type' : 'hamann',
                 'xc'                   : xc },             
         'H' : { 'gen_basis'            : True,
                 'basis_size'           : basis_size,
                 'pseudopotential_type' : 'hamann',
                 'xc'                   : xc },
         'C' : { 'gen_basis'            : True,
                 'basis_size'           : basis_size,
                 'pseudopotential_type' : 'hamann',
                 'xc'                   : xc },         
          }

In [ ]:
os.environ['ASE_CONQUEST_COMMAND'] = 'mpirun -np 4 /opt/chemistry/conquest/dev/bin/Conquest'
os.environ['CQ_PP_PATH'] = '/opt/chemistry/conquest/dev/pseudo-and-pao/'
os.environ['CQ_GEN_BASIS_CMD'] = 'mpirun -np 10 /opt/chemistry/conquest/dev/bin/MakeIonFiles'

In [ ]:
def set_Calculator_CQ(struct, basis_size, fxc, cutoff, kpoints):
    from ase.calculators.conquest import Conquest
    working_directory = mk_workdir(struct)
    
    basis = set_Basis(basis_size,fxc)
    
    conquest_flags = {'IO.WriteOutToASEFile': True}

    # Flags for atomic positions optimisation
    conquest_flags.update({'AtomMove.TypeOfRun'   : 'static',  # optimization algorithm
                           'AtomMove.MaxForceTol' :  5e-3,   # max Force component in Ha/bohr                       
                           'AtomMove.ReuseDM'     :  True,    
                           'AtomMove.AppendCoords':  True,
                           })    

    calc = Conquest(directory      = working_directory,
                    grid_cutoff    = cutoff,
                    scf_tolerance=1.0e-7,  # Ha
                    self_consistent= True,
                    xc    = fxc,
                    basis = basis,
                    kpts  = kpoints,
                    nspin = 1,
                    **conquest_flags)

    struct.calc = calc
    return(struct)

In [ ]:
def myCQget_energy(struct):
    struct = set_Calculator_CQ(struct,
                               'medium',
                               'PBE',
                               cutoff = 100.0,
                               kpoints = [8,8,8]
                              )
    
    dft_energy = struct.get_potential_energy()
    print(dft_energy, 'eV')
    return(dft_energy)

In [ ]:
import time
from ase.build import bulk

out_file = 'myOPT_en_vol'
if ( os.path.isfile('Cu_opt/'+out_file) ):
        os.remove('Cu_opt/'+out_file)
        
a =3.65 #Angstrom, set initial lattice constant

num_rep = 3 #Set the number of calculation
multis_a = range(-num_rep, num_rep+1)

t1 = time.time()
for multi_a in multis_a:
    # Set lattis constant
    a_tmp = a*(1+0.05*multi_a) # example of 5% difference
    
    # Set initial structure
    print('calc with a = ', '{:.3f}'.format(a_tmp))
    struct = bulk('Cu', 'fcc', a=a_tmp, cubic=True) 
    
    # Set working directory
    working_directory = 'cu_bulk_data/cu_bulk_cell{:.3f}'.format(struct.cell[0,0])

    # Test if `working_directory` exists ? If not create it
    if ( not os.path.isdir(working_directory) ):
    os.makedirs(working_directory)
    
    # Start calculation
    dft_energy = myCQget_energy(struct)
    
    # calculation speed check
    with open(working_directory+'/Conquest_out', 'r') as f:
        data = f.readlines()
    calc_time = float(data[-1].split()[4])
    print('calc_time = {:.3f} sec'.format(calc_time))
    
    # energy output file
    with open(working_directory+'/../Cu_opt/myOPT_en_vol', 'a') as f:
        f.write((str('{:.3f}'.format(float(struct.cell[0,0])**3)) + ' ' + str(dft_energy) + '\n'))
    print()
    
t2 = time.time()
time_elapsed = t2 - t1
print('Total_time_elapsed = {:.3f} sec'.format(time_elapsed))

In [ ]:
import matplotlib.pyplot as plt

with open('Cu_opt/myOPT_en_vol', 'r') as f:
    data = f.read()
    data = data.split('\n')

x = []
y = []
for row in data:
    #print(row.split(' '))
    if row == '':
        continue
    x.append(float(row.split(' ')[0]))
    y.append(float(row.split(' ')[1]))

plt.plot(x, y, marker = 'o')
#plt.savefig(working_directory)
plt.show()